<a href="https://colab.research.google.com/github/Jason-Durr/Projects-in-Machine-Learning/blob/main/HW/Homework3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 1

In order to use Tensorflow to create my two layer neural network is used the following documentation:

https://www.tensorflow.org/tutorials/quickstart/beginner. I used this to get the basics down for how to organize a model.

https://www.tensorflow.org/api_docs/python/tf/keras/layers/InputLayer. I referred to this documentation to determine how to size the input layer apropreatly for the arguments

https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy I used this documentation in order to understand how the loss function is meant to be called.

https://www.tensorflow.org/tutorials/keras/keras_tuner.  This documentation was used to understand how to perform a search for parameters.




In [3]:
import tensorflow as tf

In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jasondurr","key":"a288e7df4fa66f235cefea7adda21090"}'}

In [5]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c titanic


  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 17.7MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 3.16MB/s]
  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 51.6MB/s]


In [6]:
import pandas as pd
import numpy as np

In [41]:
data = pd.read_csv('train.csv')

from sklearn.model_selection import train_test_split

data, testData = train_test_split(data, test_size=0.2)

data, devData = train_test_split(data, test_size=0.2)


In [32]:
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
273,274,0,1,"Natsch, Mr. Charles H",male,37.0,0,1,PC 17596,29.7000,C118,C
505,506,0,1,"Penasco y Castellana, Mr. Victor de Satode",male,18.0,1,0,PC 17758,108.9000,C65,C
485,486,0,3,"Lefebre, Miss. Jeannie",female,NaN,3,1,4133,25.4667,NaN,S
572,573,1,1,"Flynn, Mr. John Irwin (""Irving"")",male,36.0,0,0,PC 17474,26.3875,E25,S
249,250,0,2,"Carter, Rev. Ernest Courtenay",male,54.0,1,0,244252,26.0000,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
398,399,0,2,"Pain, Dr. Alfred",male,23.0,0,0,244278,10.5000,NaN,S
693,694,0,3,"Saad, Mr. Khalil",male,25.0,0,0,2672,7.2250,NaN,C
147,148,0,3,"Ford, Miss. Robina Maggie ""Ruby""",female,9.0,2,2,W./C. 6608,34.3750,NaN,S
51,52,0,3,"Nosworthy, Mr. Richard Cater",male,21.0,0,0,A/4. 39886,7.8000,NaN,S


In [45]:

X = pd.DataFrame()
X["Survived"] = data["Survived"]
X["Sex"] = (data['Sex'] == 'male').astype(int)
X["Age"] = ((data["Age"]-data["Age"].mean())/data["Age"].std()).fillna(0)
X["Fare"] = (data["Fare"]-data["Fare"].mean())/data["Fare"].std()
X["Parch"] = data["Parch"]
X = pd.concat([X,pd.get_dummies(data.Pclass, prefix='Class')],axis=1)
X = pd.concat([X,pd.get_dummies(data.Embarked, prefix='Embarked')],axis=1)



x_test = pd.DataFrame()
y_test = np.matrix(testData['Survived']).T
x_test["Sex"] = (testData['Sex'] == 'male').astype(int)
x_test["Age"] = ((testData["Age"]-data["Age"].mean())/data["Age"].std()).fillna(0)
x_test["Fare"] = (testData["Fare"]-data["Fare"].mean())/data["Fare"].std()
x_test["Parch"] = testData["Parch"]
x_test = pd.concat([x_test,pd.get_dummies(testData.Pclass, prefix='Class')],axis=1)
x_test = pd.concat([x_test,pd.get_dummies(testData.Embarked, prefix='Embarked')],axis=1)

x_dev = pd.DataFrame()
y_dev = np.matrix(devData['Survived']).T
x_dev["Sex"] = (devData['Sex'] == 'male').astype(int)
x_dev["Age"] = ((devData["Age"]-data["Age"].mean())/data["Age"].std()).fillna(0)
x_dev["Fare"] = (devData["Fare"]-data["Fare"].mean())/data["Fare"].std()
x_dev["Parch"] = devData["Parch"]
x_dev = pd.concat([x_dev,pd.get_dummies(devData.Pclass, prefix='Class')],axis=1)
x_dev = pd.concat([x_dev,pd.get_dummies(devData.Embarked, prefix='Embarked')],axis=1)

X.corr()["Survived"]

Survived      1.000000
Sex          -0.561382
Age          -0.034528
Fare          0.240106
Parch         0.038823
Class_1       0.258838
Class_2       0.110325
Class_3      -0.313332
Embarked_C    0.113926
Embarked_Q    0.031182
Embarked_S   -0.128744
Name: Survived, dtype: float64

In [46]:
X.isna().sum()

Survived      0
Sex           0
Age           0
Fare          0
Parch         0
Class_1       0
Class_2       0
Class_3       0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64

In [47]:
X.describe()

,Survived,Sex,Age,Fare,Parch,Class_1,Class_2,Class_3,Embarked_C,Embarked_Q,Embarked_S
count,569.000000,569.000000,5.690000e+02,5.690000e+02,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,0.370826,0.655536,-6.321832e-17,5.447702e-16,0.379613,0.247803,0.198594,0.553603,0.175747,0.079086,0.741652
std,0.483451,0.475611,8.910747e-01,1.000000e+00,0.834979,0.432117,0.399293,0.497556,0.380940,0.270111,0.438111
min,0.000000,0.000000,-2.062607e+00,-6.967311e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,-4.955895e-01,-5.218254e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000e+00,-3.755309e-01,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
75%,1.000000,1.000000,3.371921e-01,-3.935577e-03,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,3.460123e+00,1.065225e+01,6.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [48]:

y_train = np.matrix(X['Survived']).T
x_train = X.drop('Survived',axis=1).to_numpy()


In [94]:
!pip install -q -U keras-tuner
import keras_tuner as kt
from tensorflow import keras

In [99]:

def model_builder(hp):

  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(10)),
    tf.keras.layers.Dense(hp_units, activation='relu'),
    tf.keras.layers.Dense(2)
  ])
  hp_learning_rate = hp.Choice('learning_rate', values=[.5e-2,1e-2, 1e-3, 1e-4])
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model




In [100]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=50,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

INFO:tensorflow:Reloading Oracle from existing project my_dir/intro_to_kt/oracle.json
INFO:tensorflow:Reloading Tuner from my_dir/intro_to_kt/tuner0.json


In [101]:
tuner.search(x_train, y_train, epochs=50, validation_data=(x_dev,y_dev), callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 128 and the optimal learning rate for the optimizer
is 0.01.



In [102]:

model = model_builder(best_hps)
model.fit(x_train, y_train, epochs=30,batch_size=40,)

Epoch 1/30
15/15 [==============================] - 0s 2ms/step - loss: 0.5504 - accuracy: 0.7100
Epoch 2/30
15/15 [==============================] - 0s 2ms/step - loss: 0.4617 - accuracy: 0.8067
Epoch 3/30
15/15 [==============================] - 0s 2ms/step - loss: 0.4367 - accuracy: 0.8067
Epoch 4/30
15/15 [==============================] - 0s 2ms/step - loss: 0.4273 - accuracy: 0.8260
Epoch 5/30
15/15 [==============================] - 0s 2ms/step - loss: 0.4062 - accuracy: 0.8313
Epoch 6/30
15/15 [==============================] - 0s 2ms/step - loss: 0.4090 - accuracy: 0.8225
Epoch 7/30
15/15 [==============================] - 0s 2ms/step - loss: 0.4129 - accuracy: 0.8207
Epoch 8/30
15/15 [==============================] - 0s 2ms/step - loss: 0.4348 - accuracy: 0.8207
Epoch 9/30
15/15 [==============================] - 0s 2ms/step - loss: 0.4051 - accuracy: 0.8260
Epoch 10/30
15/15 [==============================] - 0s 2ms/step - loss: 0.4029 - accuracy: 0.8278
Epoch 11/30
15/15 [

In [103]:
model.evaluate(x_train,  y_train, verbose=2)

18/18 - 0s - loss: 0.3364 - accuracy: 0.8594


[0.33637821674346924, 0.8594024777412415]

In [104]:
model.evaluate(x_dev,  y_dev, verbose=2)

5/5 - 0s - loss: 0.3930 - accuracy: 0.8601


[0.39295658469200134, 0.8601398468017578]

In [106]:
model.evaluate(x_test,  y_test, verbose=2)

6/6 - 0s - loss: 0.4693 - accuracy: 0.7877


[0.46934303641319275, 0.7877094745635986]

# Task 3

To select the hyper parameters I performed a grid search for the number of nodes in the second layer and the step size and my results found that 128 nodes and a step size of .01 performed the best for the dev results.  I did not use regularization because I felt that given the result from the training versus the dev set there was not too much overfitting thanks to the way that we searched for hyper parameters.  I chose to use the Adams optimizer because it was able to produce good results quickly and so I felt it worked the best.